In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('watermark').master('local[2]').getOrCreate()

In [3]:
spark

In [4]:
df = spark.readStream.format('socket').option('host','localhost')\
.option('port',9999).load()

In [5]:
df.printSchema()

root
 |-- value: string (nullable = true)



In [6]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [7]:
df1 = df.select(split('value',',').alias('data'))

In [8]:
df1.printSchema()

root
 |-- data: array (nullable = true)
 |    |-- element: string (containsNull = false)



In [9]:
df2 = df1.select(df1.data[0].cast(TimestampType()).alias('ts'),df1.data[1].alias('name'))

In [10]:
df2.printSchema()

root
 |-- ts: timestamp (nullable = true)
 |-- name: string (nullable = true)



In [11]:
#res = df2.groupby(window('ts','10 minutes'),'name').count()

In [12]:
#res.printSchema()

In [13]:
#res= res.select("window.start","window.end","name","count")

In [14]:
#res.printSchema()

In [15]:
#res.writeStream.format('console').outputMode('update').trigger(processingTime="5 minutes").start().awaitTermination()

In [ ]:
res = df2.withWatermark('ts','2 minutes').groupby(window('ts','2 minutes'),'name').count()

In [17]:
res.printSchema()

root
 |-- window: struct (nullable = false)
 |    |-- start: timestamp (nullable = true)
 |    |-- end: timestamp (nullable = true)
 |-- name: string (nullable = true)
 |-- count: long (nullable = false)



In [ ]:
res= res.select("window.start","window.end","name","count")

In [ ]:
res.writeStream.format('console').outputMode('append').trigger(processingTime="90 seconds").start().awaitTermination()